In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import zipfile
import os

def unzip_files(zip_file_paths, output_folder):
    for zip_file_path in zip_file_paths:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(output_folder)

if __name__ == "__main__":
    # Replace these paths with the actual paths of your ZIP files
    zip_file_paths = ["/content/drive/MyDrive/sign dataset/val_raw_videos.zip", "/content/drive/MyDrive/sign dataset/val_rgb_side_raw_videos.zip"]

    # Replace this path with the desired output folder for extracted files
    output_folder = "/content/drive/MyDrive/sign dataset/output_videos"

    unzip_files(zip_file_paths, output_folder)


In [ ]:
import cv2
import pandas as pd
import os
import shutil

def extract_frames(video_path, start_time, end_time, output_folder):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Set the start and end frame positions
    start_frame = int(start_time * cap.get(cv2.CAP_PROP_FPS))
    end_frame = int(end_time * cap.get(cv2.CAP_PROP_FPS))

    # Set the current frame position to the start frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Loop through the frames and save them as images
    for frame_num in range(start_frame, end_frame):
        ret, frame = cap.read()
        if not ret:
            break
        frame_name = f"{output_folder}/frame_{frame_num}.jpg"
        cv2.imwrite(frame_name, frame)

    # Release the video capture object
    cap.release()

def process_videos(csv_file, videos_folder, output_folder, max_folder_length=100):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file, delimiter='\t')

    # Iterate over rows and extract frames for each video
    for index, row in df.iterrows():
        video_name = row['VIDEO_NAME']
        start_time = row['START']
        end_time = row['END']
        sentence = row['SENTENCE']

        # Truncate folder name if it exceeds max_folder_length
        sentence_folder = f"{sentence[:max_folder_length]}_frames"
        os.makedirs(sentence_folder, exist_ok=True)

        # Extract frames for the current video and save in the corresponding folder
        video_path = os.path.join(videos_folder, f"{video_name}.mp4")
        extract_frames(video_path, start_time, end_time, sentence_folder)

        # Move the sentence folder to the 'annotations' folder using shutil.move
        annotation_folder = os.path.join(output_folder, "annotations")
        os.makedirs(annotation_folder, exist_ok=True)
        shutil.move(sentence_folder, os.path.join(annotation_folder, sentence_folder))

if __name__ == "__main__":
    csv_file_path = "/content/drive/MyDrive/sign dataset/how2sign_val.csv"  # Update the path to your CSV file
    videos_folder_path = "/content/drive/MyDrive/sign dataset/output_videos/raw_videos"     # Update the path to your videos folder
    annotations_folder_path = "/content/drive/MyDrive/sign dataset/annotations"  # Update the path to your desired annotations folder
    process_videos(csv_file_path, videos_folder_path, annotations_folder_path)


In [ ]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 47.4 MB/s eta 0:00:00


In [ ]:
import os
import pickle
import mediapipe as mp
import cv2

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Update the path to your Google Drive directory
DATA_DIR = '/content/drive/MyDrive/sign dataset/annotations/annotations'

for dir_ in os.listdir(DATA_DIR):
    dir_path = os.path.join(DATA_DIR, dir_)

    if not os.path.isdir(dir_path):
        continue

    data = []
    labels = []

    for img_path in os.listdir(dir_path):
        data_aux = []

        x_ = []
        y_ = []

        img = cv2.imread(os.path.join(dir_path, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            data.append(data_aux)
            labels.append(dir_)

    # Save the data and labels into a pickle file for each annotation folder
    output_file = os.path.join('/content/drive/MyDrive/sign dataset', 'data.pickle')  # Update the path
    with open(output_file, 'wb') as f:
        pickle.dump({'data': data, 'labels': labels}, f)
